In [1]:
import pandas as pd
import numpy as np
import os
import sys
currentdir = os.getcwd() 
parent_dir = os.path.dirname(currentdir)
sys.path.insert(0, parent_dir) 
sys.path.insert(0,'../../utils')
import ast
import json
from df_utils import *
from helper_solution_analysis import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Load Model, Participant solutions and the solution space

In [2]:
# get data csv
par_parent_dir = os.path.dirname(parent_dir)
data_file = 'data/full_experiment/data_furnituremanufacturing_2021-08-11_09-15.csv'
data_path = os.path.join(par_parent_dir, data_file)

# transform to dataframe
df, df_model, df_ge, df_vp_sol, df_qual,  df_actions, df_built, df_resources, df_play_times = decompose_csv(data_path)
df_complete_sol = get_complete_sol(df_resources)
resources_month, rewards_month, cost_dict = load_lp_model()
_, _, leaves, _ = get_sol_leaves()


read cache


In [3]:
# reformat the complete solution frame
dfc = df_complete_sol.melt(value_name = 'complete', ignore_index=False)


In [4]:
# dictionary for all solutions and solution trajectories
solution_dictionary = {}
for i in range(12):
    vp = dfc.index[(dfc.month == i+1)]
    solution_dictionary[i] = solution_dict(vp, i+1,df_built)

# dictionary for only the complete solutions
solution_dict_complete ={}
for i in range(12):
    vp = dfc.index[(dfc.month == i+1) & (dfc.complete == True)]
    solution_dict_complete[i] = solution_dict(vp, i+1,df_built)

# map all the alternative solutions in the solution space to profits
leaves_profit = []
for i in range(12):
    l = []
    rewards = rewards_month[i]
    for j in leaves[i]:
        profit = j[0]*rewards[0] + j[1]*rewards[1] + j[2]*rewards[2] + j[3]*rewards[3]
        l.append((j,profit))
    leaves_profit.append(l)

    

def profit_dict(prices, solution):
    # mapping from the different steps in the solution to profit,
    # one dict entry for each vp with all the monthly solutions
    profit_graph = {}
    for k in solution.keys():
        profit_graph[k] = []
        for i in solution[k]:
            profit = round(i[0]*prices[0] + i[1]*prices[1] + i[2]*prices[2] + i[3]*prices[3])
            profit_graph[k].append(profit)
    return profit_graph


def profit_dict_per_vp(prices, solution_dictionary, vp):
    # mapping from the different steps in the solution to profit,
    #one dict entry for each month with all the vp solutions
        profit_graph = {}
        for m in range(12):
            profit_graph[m] = []
            for i in solution_dictionary[m][vp]:
                profit = round(i[0]*prices[m][0] + i[1]*prices[m][1] + i[2]*prices[m][2] + i[3]*prices[m][3])
                profit_graph[m].append(profit)
        return profit_graph

In [5]:
for k in range(12):
    solution = solution_dict_complete[k]
    profit_graph = profit_dict(rewards_month[k],solution)
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Profit reached by each vp per timestep in month ' +str(k+1), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in profit_graph.items() ]))
    df.columns.name = 'VP'
    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('Profit', fontsize = 17)
    fig.savefig('Images/optimal_paths/optimal_paths_' + str(k+1) + '.jpg' ,bbox_inches='tight')
    plt.close()

In [6]:

for vp in dfc.index.unique():
    profit_graph = profit_dict_per_vp(rewards_month, solution_dictionary, vp)
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Profit reached by each month per vp ' +str(vp), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in profit_graph.items() ]))
    df.columns.name = 'VP'

    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('Profit', fontsize = 17)
    fig.savefig('Images/optimal_paths/optimal_paths_vp_' + str(vp) + '.jpg' ,bbox_inches='tight')
    plt.close()

In [7]:

def pos_solutions_dict(vp_list, month, solution):
    # number of reachable solutions from each step for all vp in one month 
    pos_solutions = {}
    for v in vp_list:
        pos_solutions[v] = []
        solutions = leaves[month -1]
        for sol in solution[v]:
            solutions = [x for x in solutions if all(i <= j for i, j in zip(sol, x))] 
            pos_solutions[v].append(len(solutions))
    return pos_solutions

def pos_solutions_dict_per_vp(vp, solution_dictionary):
    # number of reachable solutions from each step for all months for one vp 
    pos_solutions = {}
    for m in range(12):
        pos_solutions[m] = []
        solutions = leaves[m]
        for sol in solution_dictionary[m][vp]:
            solutions = [x for x in solutions if all(i <= j for i, j in zip(sol, x))]
            pos_solutions[m].append(len(solutions))
    return pos_solutions

def best_solutions_dict(vp_list, month, solution):
    # best profit still possible for each vp in one month
    best_solutions = {}
    for v in vp_list:
        best_solutions[v] = []
        solutions = leaves_profit[month]
        
        for sol in solution[v]:
     
            profits = [y for x,y in solutions if all(i <= j for i, j in zip(sol, x))]
            best_solutions[v].append(max(profits))
    return best_solutions



def best_solutions_dict_per_vp(vp, solution_dictionary):
    # best profit still possible for all months for one vp

    best_solutions = {}
    for m in range(12):
        best_solutions[m] = []
        solutions = leaves_profit[m]
        for sol in solution_dictionary[m][vp]:
            profits = [y for x,y in solutions if all(i <= j for i, j in zip(sol, x))]
            best_solutions[m].append(max(profits))
    return best_solutions

In [8]:

for k in range(12):
    vp = dfc.index[(dfc.month == k+1) & (dfc.complete == True)]
   
    solution_graph = pos_solutions_dict(vp, k, solution_dict_complete[k])
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Amount of possible solutions ' +str(k+1), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in solution_graph.items() ]))
    df.columns.name = 'Month'

    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('solutions', fontsize = 17)
    fig.savefig('Images/solution_numbers/solution_numbers_' + str(k+1) + '.jpg' ,bbox_inches='tight')
    plt.close()

In [9]:
for vp in dfc.index.unique():
    solution_graph = pos_solutions_dict_per_vp(vp, solution_dictionary)
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Amount of possible solutions for vp ' +str(vp), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in solution_graph.items() ]))
    df.columns.name = 'Month'

    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('solutions', fontsize = 17)
    fig.savefig('Images/solution_numbers_per_vp/solution_numbers_' + str(vp) + '.jpg' ,bbox_inches='tight')
    plt.close()

In [10]:
for k in range(12):
    vp = dfc.index[(dfc.month == k+1) & (dfc.complete == True)]

    solution = solution_dict_complete[k]
    profit_graph = best_solutions_dict(vp, k, solution)
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Profit possible by each vp per timestep in month ' +str(k+1), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in profit_graph.items() ]))
    df.columns.name = 'VP'

    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('Profit', fontsize = 17)
    fig.savefig('Images/best_profit/best_profit_' + str(k+1) + '.jpg' ,bbox_inches='tight')
    plt.close()

In [11]:
for vp in dfc.index.unique():
    solution_graph = best_solutions_dict_per_vp(vp, solution_dictionary)
    fig = plt.figure(figsize=(23, 15) )
    plt.title('Amount of possible solutions for vp ' +str(vp), fontsize = 25)

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in solution_graph.items() ]))
    df.columns.name = 'Month'

    g =sns.lineplot(data = df)
    xlabel = plt.xlabel('amount furniture', fontsize = 17)
    ylabel = plt.ylabel('solutions', fontsize = 17)
    fig.savefig('Images/best_profit/best_profit_vp_' + str(vp) + '.jpg' ,bbox_inches='tight')
    plt.close()